# Titanic - Machine Learning from Disaster

Легендарное ML-соревнование с [Kaggle](https://www.kaggle.com/c/titanic)

<img src='./Titanic.jpg' width=500>

Возможно, гибель Титаника - одно из самых печально известных кораблекрушений в истории. Титаник был крупнейшим действующим океанским лайнером своего времени, у него были улучшенные меры обеспечения безопасности, такие как водонепроницаемые отсеки и водонепроницаемые двери с дистанционным управлением. Корабль считался «непотопляемым», однако он затонул рано утром 15 апреля 1912 года в северной части Атлантического океана во время своего первого рейса из Саутгемптона в Нью-Йорк. В момент столкновения корабля с айсбергом на борту находились 2224 человека.

В соответствии с существующей в то время практикой система спасательных шлюпок Титаника была разработана для переправки пассажиров на ближайшие спасательные суда, а не для одновременного удержания всех на плаву. Таким образом, когда корабль быстро тонул (корабль затонул за 2 часа 40 минут), а до прибытия помощи оставалось еще несколько часов, многим пассажирам и членам экипажа не хватило места на спасательных шлюпках, которых было всего 20. Плохая организация эвакуации привела к тому, что многие лодки были спущены на воду до того, как были полностью заполнены.

В результате кораблекрушения погибло более 1500 человек, что сделало его одним из самых смертоносных в истории.

Без сомнения, при спасении пассажиров имел место элемент удачи, но, возможно, были те, кто имели большие шансы уцелеть чем другие. [Titanic ML competition on Kaggle](https://www.kaggle.com/c/titanic) предлагает участникам предсказать кто их пассажиров пережил кораблекрушение, основываясь на сохранившихся данных о пассажирах.

В Интернете можно найти десятки статей, посвященных этому соревнованию, и сотни решений этой задачи. В исследовательском анализе я во многом ориентировался на [эту статью](https://habr.com/ru/company/mlclass/blog/270973/) и позаимствовал из неё несколько идей.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-модулей-и-константы" data-toc-modified-id="Импорт-модулей-и-константы-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт модулей и константы</a></span></li><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Примечания-к-признкакм" data-toc-modified-id="Примечания-к-признкакм-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Примечания к признкакм</a></span></li><li><span><a href="#Пути-к-файлам" data-toc-modified-id="Пути-к-файлам-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Пути к файлам</a></span></li><li><span><a href="#Предположения" data-toc-modified-id="Предположения-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Предположения</a></span></li><li><span><a href="#Анализ-таблицы-с-результатами-соревнований" data-toc-modified-id="Анализ-таблицы-с-результатами-соревнований-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Анализ таблицы с результатами соревнований</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Разделения-на-тестовый-и-тренировочный-наборы" data-toc-modified-id="Разделения-на-тестовый-и-тренировочный-наборы-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Разделения на тестовый и тренировочный наборы</a></span></li><li><span><a href="#Исследовательский-анализ" data-toc-modified-id="Исследовательский-анализ-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Исследовательский анализ</a></span><ul class="toc-item"><li><span><a href="#Предварительный-анализ" data-toc-modified-id="Предварительный-анализ-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Предварительный анализ</a></span></li><li><span><a href="#Детальный-анализ-признаков" data-toc-modified-id="Детальный-анализ-признаков-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Детальный анализ признаков</a></span><ul class="toc-item"><li><span><a href="#Pclass" data-toc-modified-id="Pclass-11.2.1"><span class="toc-item-num">11.2.1&nbsp;&nbsp;</span>Pclass</a></span></li><li><span><a href="#Name" data-toc-modified-id="Name-11.2.2"><span class="toc-item-num">11.2.2&nbsp;&nbsp;</span>Name</a></span></li><li><span><a href="#Sex" data-toc-modified-id="Sex-11.2.3"><span class="toc-item-num">11.2.3&nbsp;&nbsp;</span>Sex</a></span></li><li><span><a href="#Age" data-toc-modified-id="Age-11.2.4"><span class="toc-item-num">11.2.4&nbsp;&nbsp;</span>Age</a></span></li><li><span><a href="#SibSp" data-toc-modified-id="SibSp-11.2.5"><span class="toc-item-num">11.2.5&nbsp;&nbsp;</span>SibSp</a></span></li><li><span><a href="#Parch" data-toc-modified-id="Parch-11.2.6"><span class="toc-item-num">11.2.6&nbsp;&nbsp;</span>Parch</a></span></li><li><span><a href="#Fare" data-toc-modified-id="Fare-11.2.7"><span class="toc-item-num">11.2.7&nbsp;&nbsp;</span>Fare</a></span></li><li><span><a href="#Cabin" data-toc-modified-id="Cabin-11.2.8"><span class="toc-item-num">11.2.8&nbsp;&nbsp;</span>Cabin</a></span></li><li><span><a href="#Embarked" data-toc-modified-id="Embarked-11.2.9"><span class="toc-item-num">11.2.9&nbsp;&nbsp;</span>Embarked</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Обработка данных</a></span><ul class="toc-item"><li><span><a href="#Разработка-новых-признаков" data-toc-modified-id="Разработка-новых-признаков-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Разработка новых признаков</a></span><ul class="toc-item"><li><span><a href="#Title" data-toc-modified-id="Title-12.1.1"><span class="toc-item-num">12.1.1&nbsp;&nbsp;</span><code>Title</code></a></span></li><li><span><a href="#AgeGroup-и-FareGroup" data-toc-modified-id="AgeGroup-и-FareGroup-12.1.2"><span class="toc-item-num">12.1.2&nbsp;&nbsp;</span><code>AgeGroup</code> и <code>FareGroup</code></a></span></li><li><span><a href="#Family" data-toc-modified-id="Family-12.1.3"><span class="toc-item-num">12.1.3&nbsp;&nbsp;</span><code>Family</code></a></span></li><li><span><a href="#isFramily-и-isCabin" data-toc-modified-id="isFramily-и-isCabin-12.1.4"><span class="toc-item-num">12.1.4&nbsp;&nbsp;</span><code>isFramily</code> и <code>isCabin</code></a></span></li></ul></li><li><span><a href="#Оценка-информативности-признаков" data-toc-modified-id="Оценка-информативности-признаков-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Оценка информативности признаков</a></span><ul class="toc-item"><li><span><a href="#Конвейер" data-toc-modified-id="Конвейер-12.2.1"><span class="toc-item-num">12.2.1&nbsp;&nbsp;</span>Конвейер</a></span></li><li><span><a href="#Корреляция-между-обучающими-признаками" data-toc-modified-id="Корреляция-между-обучающими-признаками-12.2.2"><span class="toc-item-num">12.2.2&nbsp;&nbsp;</span>Корреляция между обучающими признаками</a></span></li><li><span><a href="#Взаимная-информация-(Mutual-Information)" data-toc-modified-id="Взаимная-информация-(Mutual-Information)-12.2.3"><span class="toc-item-num">12.2.3&nbsp;&nbsp;</span>Взаимная информация (Mutual Information)</a></span></li></ul></li></ul></li><li><span><a href="#Базовая-модель" data-toc-modified-id="Базовая-модель-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Базовая модель</a></span></li><li><span><a href="#Выбор-модели" data-toc-modified-id="Выбор-модели-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Выбор модели</a></span></li><li><span><a href="#Отбор-признаков" data-toc-modified-id="Отбор-признаков-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Отбор признаков</a></span></li></ul></div>

## Импорт модулей и константы

In [ ]:
import warnings


from category_encoders.target_encoder import TargetEncoder

from imblearn.over_sampling import SMOTE

from imblearn.pipeline import Pipeline

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from pandas.plotting import scatter_matrix

import seaborn as sns

from scipy.stats import (percentileofscore,
                         randint,
                         ttest_ind,
                         uniform,
                        )

from sklearn.compose import ColumnTransformer

from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier

from sklearn.ensemble import (GradientBoostingClassifier, 
                              RandomForestClassifier)

from sklearn.feature_selection import (mutual_info_classif, 
                                       RFE,
                                       RFECV)

from sklearn.impute import (MissingIndicator, 
                            SimpleImputer)

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (get_scorer, 
                             roc_curve)

from sklearn.model_selection import (cross_val_predict, 
                                     cross_val_score,
                                     train_test_split,)

from sklearn.preprocessing import (Binarizer, 
                                   FunctionTransformer, 
                                   KBinsDiscretizer, 
                                   StandardScaler)

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier


warnings.simplefilter(action='ignore', category=FutureWarning)


SEED = 42
CV = 10


%matplotlib inline

## Описание проекта

**Цель** - предсказать выживет пассажир Тинтаника в кораблекрушении или нет.

**Тип задачи** - классификация (бинарная классификация).

**Целевая метрика** - *accuracy* (точность).

**Желаемое значение целевой метрики:** Табилца с результатами соревнований содержит почти 14000 записей. 
Медианное значение точности 0.775, но менее 4% имеют резутат выше 0.8. 
Таким образом, **значение метрики accuracy больше или равное 0.8 будет отличным результатом**.

**Существующие решения:** имеется огромное количество решений опубликованных в интернете и [на форуме](https://www.kaggle.com/c/titanic/discussion)

## Описание данных
0. **PassengerId** - ID пассажира.
1. **Survived** - Выживший:
    - 0 = Нет, 
    - 1 = Да.
2. **Pclass** - Класс билета:
    - 1 = первый, 
    - 2 = второй, 
    - 3 = третий.
3. **Name** - Имя пассажира, например Braund, Mr. Owen Harris.
4. **Sex** - Пол:
    - male - мужчина,
    - female - женщина.
5. **Age** - Возраст в годах, например 38.0.
6. **SibSp** - Количество братьев и сестер или супругов на борту Титаника.
7. **Parch** - Количество родителей или детей на борту Титаника.
8. **Ticket** - Номер билета, например A/5 21171.
9. **Fare** - Стоимость билета, например 71.2833.
10. **Cabin** - Номер каюты, напрмер C85.
11. **Embarked** - Порт, где пассажир сел на корабль:
    - C = Cherbourg,
    - Q = Queenstown,
    - S = Southampton.
    
**Признаки:** PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket,
Fare, Cabin, Embarked

**Целевой признак** - Survived
    
## Примечания к признкакм
- **pclass**: социально-экономический статус
    - *1st* = Upper
    - *2nd* = Middle
    - *3rd* = Lower
- **age**: Возраст представлен дробным значением, если он меньше 1 года. Если возрас известен, то значение имеет вид xx.5.
- **sibsp**: Количество братьев и сестер или супругов на борту Титаника
    - *sibling* = брат, сестра, сводный брат, сводная сестра
    - *spouse* = муж, жена (любовницы, женихи и невесты игнорировались)
- **parch**: количество родителей (мать, отец)/детей (дочь, сын, в т.ч. приёмные) на борту Титаника. Некоторые дети путешествовали только с няней, поэтому для них parch=0.

## Пути к файлам
- **training set**: ../datasets/train.csv - набор данных для обучения модели.
- **test set**: ../datasets/test.csv - набор для подготовки предсказания для отправки на проверку.
- **example of a submission file**: ../datasets/gender_submission.csv - пример данных, подготовленных для проверки. В нём значения целевой переменной расставлены случайным образом.

## Предположения
У женщин было больше шансов выжить, чем у мужчин.

## Анализ таблицы с результатами соревнований
Скачаем [таблицу с результатами соревнования](https://www.kaggle.com/competitions/titanic/leaderboard) и проанализируем результаты в ней. Таблица загружена 07.01.2023.

Выведем первые 5 строк таблицы.

In [ ]:
leaderboard = pd.read_csv('../datasets/titanic-publicleaderboard.csv')
leaderboard.head()

Неожиданно, в начале таблицы оказались результаты с точностью (метрика accuracy) равной 1. Неужели авторы смогли достичь совершенства?

Посчитаем описательные статистики для результатов приведённых в таблице (столбец `Score`).

In [ ]:
leaderboard.Score.describe()

Посчитаем процент участников показавших результат лучше 0.8.

In [ ]:
print(f'Только {1 - percentileofscore(leaderboard.Score, 0.8) / 100 :.2%} участников показали результат лучше 0.8')

Таким образом, **значение метрики accuracy больше или равное 0.8 будет отличным результатом**.

Построим функцию распределения (ECDF) вероятности результатов участников. Красным отметим желаемое значение целевой метрики.

In [ ]:
def plot_ecdf_with_target(data, target):
    sns.displot(data, stat='proportion', kind='ecdf', height=5, aspect=1)
    
    quantile = percentileofscore(data, target) / 100
    
    plt.plot([0, target, target], [quantile, quantile, 0], '-.r')
    plt.plot([target], [quantile], 'or')
    
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    
    plt.title('ECDF результатов соревнования')
    
    plt.grid()
    
    plt.show()

    
plot_ecdf_with_target(leaderboard.Score, 0.8)

Построим плотность распределения вероятности получения определённого результата соревнования. Красной стрелкой отметим результаты равные 1.0 .

In [ ]:
sns.displot(leaderboard, x='Score', kind='kde')

plt.arrow(x=1.0, y=5, dx=0, dy=-3.5, width=0.01, head_width=0.04, head_length=0.8, color='r')
plt.title("KDE результатов соревнования")
plt.grid()

plt.show()

Возможно, решения с "совершенным" результатом появились из-за того, что на [GitHub](https://github.com/thisisjasonjafari/my-datascientise-handcode/raw/master/005-datavisualization/titanic.csv) есть точное решение задачи соревнования. Вполен возможно, что выложены данные, извлечённые из [Encyclopedia Titanica](https://www.encyclopedia-titanica.org/titanic-survivors/) или из [OpenML](https://www.openml.org/search?type=data&sort=runs&id=40945&status=active).

Некоторые авторы в своих тетрадках честно предупреждают других пользователей о наличии такой возможности, например [вот этот](https://www.kaggle.com/code/suzukifelipe/how-to-be-a-top-lb-explained-for-beginners/notebook?scriptVersionId=99817039).

Приступим к решению задачи.

## Загрузка данных
Загрузим набор данных для разработки модели. Далее будем работать с этим набором данных.

In [ ]:
data = pd.read_csv('../datasets/train.csv')

Загрузим набор признкаов для отправки результатов на проверку. Отложим этот набор.

In [ ]:
submission_X = pd.read_csv('../datasets/test.csv')

## Знакомство с данными
Выведем форму датасета.

In [ ]:
print(f'Датасет содержит {data.shape[0]} строку и {data.shape[1]} столбцов.')

Выведем на экран первые 10 строк датасета

In [ ]:
data.head(10)

Проверим типы данных признаков и наличие пропусков

In [ ]:
data.info()

Столбцы `Age`, `Cabin`, `Embarked` содержат пропуски, причём, в столбце `Cabin` пропусков большинство.

Проверим есть ли дисбаланс классов целевой переменной.

In [ ]:
def check_target_imbalance(vals):
    mean_val = vals.mean()
    
    print(f'Доля выживших пассажиров - {mean_val: .2%}')
    print(f'Доля погибших пассажиров - {1 - mean_val: .2%}')
    
check_target_imbalance(data.Survived)

В датасете есть дисбаланс классов, поэтому будем использовать стратификацию при разделении на тренировочный и тестовый наборы и апсемплинг при обучении модели.

Проверим наличие дубликатов в наборе данных.

In [ ]:
if data.duplicated().any():
    print('В данных есть дубликаты!')
else:
    print('Дубликаты не обнаружены.')

## Разделения на тестовый и тренировочный наборы
Чтобы обучить модель и спрогнозировать точность её предсказаний на новых данных (результат соревнования) необходимо разделить данные (датафрейм `data`) на тестовый и тренировочный наборы. При разделении будем делать стратификацию по целевому признаку.

In [ ]:
X = data.drop(columns = 'Survived')
y = data.Survived.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

Проверим дисбаланс классов в тренировочной выборке

In [ ]:
check_target_imbalance(y_train)

Проверим дисбаланс классов в тестовой выборке

In [ ]:
check_target_imbalance(y_test)

Пропорции приблизительно одинаковые.

## Исследовательский анализ

### Предварительный анализ

Распечатаем описательные статистики для числовых признаков

In [ ]:
X_train.describe()

В таблицу попал категориальный признак `Pclass` рассмотрим его отдельно

In [ ]:
X_train[['Pclass']].astype('object').describe()

Выведем описательные статистики для столбцов с типом `object`.

In [ ]:
X_train.describe(include='object')

Признак `PassengerID` - уникальный идентификатор пассажира, он не несёт никакой информации. 

Признаки `Name` и `Ticket` имеют слишком большое количество уникальных значений и, скорее всего, не помогут нам. Однако, из признака `Name` можно извлечь титул, который принадлежал человеку, нёс информацию о его социально-экономическом положении и, чисто теоретически, мог влиять на шанс выжить.

Признак `Cabin` содержит большое количество пропущенных значений, тем не менее, можно попробовать предположить, что номер каюты известен для выживших пассажиров, и закодировать номер каюты единицей, когда он известен, а когда не известен, закодировать его нулём, или применить другой способ кодирования категориальной переменной.

Выведем гистограммы потенциально полезных количественных признаков.

In [ ]:
num_columns = ['Age', 'SibSp', 'Parch', 'Fare']

X_train[num_columns].hist(figsize=(7,7))
plt.show()

Распределение пассажиров по возрастам  близко к нормальному, остальные по форме далеки от нормального распределения. Также признаки имеют очень разные масштабы.

Проверим наличие корреляции между количественными признакми.

In [ ]:
scatter_matrix(X_train[num_columns],
               figsize=(7, 7),
               alpha=0.2,
              )

plt.show()

Построим матрицу корреляции

In [ ]:
def plot_corr_matrix(df, size=(7, 7), vmin=-1, vmax=1, method='pearson'):
    corr = df.corr(method=method)
    
    mask = np.triu(np.ones_like(corr, dtype=bool))
    
    f, ax = plt.subplots(figsize=size)
    
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(corr,
                mask=mask,
                cmap=cmap,
                annot=True,
                vmax=vmax,
                vmin=vmin,
                center=0.0,
                square=True,
                linewidths=1.0,
                cbar_kws={'shrink': 0.5},
                ax=ax
               )
    
    ax.set_title('Матрица корреляции')
    
    plt.show()
    
plot_corr_matrix(X_train[num_columns], vmin=-0.4, vmax=0.4)

Наблюдается небольшая отрицательная корреляция между признакми `SibSp` и `Age`, коэффициент корреляции равер -0.31, и небольшая положительная корреляция между `Parch` и `SibSp`, коэффициент корреляции равен 0.39.

### Детальный анализ признаков
Рассмотрим каждый признак по отдельности, чтобы найти перспективные преобразования для них и выявить аномалии в данных. Признаки `PassengerId` и `Ticket` рассматривать не будем.

Для этого снова присоединим к набору целевой признак и присвоем их переменной `exploratory_set`.

In [ ]:
exploratory_set = pd.concat([X_train, y_train], axis='columns')
exploratory_set.head()

#### Pclass
Посчитаем количество пассажиров для каждого класса билета.

In [ ]:
sns.catplot(data=exploratory_set, x='Pclass', kind='count')

plt.title('Количество пассажиров в каждом классе')
plt.xlabel('Класс билета')
plt.ylabel('Количество человек')

plt.show()

display(exploratory_set.Pclass.value_counts())

Больше всего пассажиров находились в третьем классе.

Оценим долю выживших для каждого класса.

In [ ]:
sns.catplot(data=exploratory_set, x='Pclass', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших в каждом классе')
plt.xlabel('Класс билета')
plt.ylabel('Доля выживших')

plt.show()

Самая высока смертность была среди пассажиров третьего класса, возможно, это было связано с тем, что их каюты находились дальше всего от спасательных шлюпок.

Оценим долю выживших для каждого класса в разрезе полов.

In [ ]:
sns.catplot(data=exploratory_set, x='Pclass', y='Survived', hue='Sex', kind='bar', errorbar=None)

plt.title('Доля выживших в каждом классе')
plt.xlabel('Класс билета')
plt.ylabel('Доля выживших')

plt.show()

Таким образом, в каждом классе выжило больше женщин, чем мужчин. Самая высокая смертность была среди мужчин третьего класса.

#### Name
Имена пассажиров, как мы заметили в предварительном анализе, все уникальны. Однако, они содержат титул, который может указывать на социально-экономический статус пассажира и может быть связан с шансами выжить в кораблекружении.

Выведем первые 5 значений признака `Name`.

In [ ]:
exploratory_set[['Name']].head()

Выделим новый признак `Title` (титул) и добиви его к `exploratory_set`, затем выведем первые 5 строк получившегося датафрейма.

In [ ]:
exploratory_set['Title'] = exploratory_set.Name.str.extract(pat=r'\b,\s(.+?)\.\s[\b(]?', expand=True)
exploratory_set['Title'] = exploratory_set['Title'].str.lower()
exploratory_set.head()

Посчитаем описательные статистики для нового признака.

In [ ]:
exploratory_set[['Title']].describe()

Определим количество повторений каждого титула в наборе данных.

In [ ]:
sns.catplot(data=exploratory_set, y='Title', kind='count')

plt.title('Титулы пассажиров')
plt.xlabel('Количество человек')
plt.ylabel('Титул')

plt.show()

display(exploratory_set.Title.value_counts(dropna=False))

Обращение *master* в то время использовалось по отношению к младенцам мужского пола. Также признак содержит несколько редко встречающихся титулов. Выведем их, затем вынесем в отдельный список `aristocratic_titles`.

In [ ]:
aristocratic_titles = pd.DataFrame(exploratory_set.Title.value_counts()).query('Title < 10')
display(aristocratic_titles)

aristocratic_titles = list(aristocratic_titles.index)

Заменим их значением *aristocratic* (аристократический).

In [ ]:
exploratory_set.loc[exploratory_set.Title.isin(aristocratic_titles), 'Title'] = 'aristocratic'

Посмотрим как владельцы разных титулов были распределены по классам билетов.

In [ ]:
sns.catplot(data=exploratory_set, y='Title', x='Pclass', hue='Survived', orient='h', alpha=0.5)

plt.title('Распреедление владельцев титулов по классам билетов')
plt.xlabel('Класс билета')
plt.ylabel('Титул')

plt.show()

Владельцы аристократических титулов занимали только первый и второй класс.

Оценим долю выживших для каждого титула. Также просматривается некоторая связь между титулом, классом и долей выживших.

In [ ]:
sns.catplot(data=exploratory_set, x='Title', y='Survived', kind='bar', errorbar=None, aspect=1.5)

plt.title('Доля выживших для каждого титула')
plt.xlabel('Титул')
plt.ylabel('Доля выживших')

plt.show()

По смотрим ту же метрику в разрезе полов.

In [ ]:
sns.catplot(data=exploratory_set, x='Title', y='Survived', hue='Sex', kind='bar', errorbar=None, aspect=1.5)

plt.title('Доля выживших для каждого титула')
plt.xlabel('Титул')
plt.ylabel('Доля выживших')

plt.show()

Среди мужчин на Титанике аристократы и младенцы имели более высокие шансы спастись.

#### Sex
Оценим количество женщин и мужчин севших на Титаник

In [ ]:
sns.catplot(data=exploratory_set, x='Sex', kind='count')

plt.title('Количество мужчин и женщин')
plt.xlabel('Пол')
plt.ylabel('Количество человек')

plt.show()

display(X_train.Sex.value_counts())

Мужчин на титанике было почти в 2 раза больше чем женщин.

Оценим количество выживших пассажиров каждого пола.

In [ ]:
sns.catplot(data=exploratory_set, x='Sex', hue='Survived', kind='count')

plt.title('Количество выживших и утонувших')
plt.xlabel('Пол')
plt.ylabel('Количество человек')

plt.show()

Оценим долю выживших среди мужчин и женщин

In [ ]:
sns.catplot(data=data, x='Sex', y='Survived', kind='bar', errorbar=None)
plt.show()

Женщины на Титанике явно имели более высоки шансы выжить, чем мужчины, что согласуется с протоколом "Женщины и дети первые", действовавшим в то время.

#### Age

Рассмотрим распределение возрастов пассажиров Титаника, для этого построим гистограмму признака `Age`.

In [ ]:
sns.displot(exploratory_set.Age, stat='density')

plt.xlabel('Возраст')
plt.ylabel('Плотность вероятности')
plt.title('Распределение возрастов пассажиров')

plt.show()

display(exploratory_set.Age.describe())

Форма распределения отклоняется от формы нормального.

Проверим гипотезу о том, что выжившие пассажиры моложе погибших. Построим распределение возрастов отдельно для погибших и отдельно для выживших.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))


sns.histplot(data=exploratory_set, x='Age', stat='density', hue='Survived', ax=ax[0])

ax[0].set_xlabel('Возраст')
ax[0].set_ylabel('Плотность вероятности')
ax[0].set_title('Распределение возрастов пассажиров')

sns.boxplot(data=exploratory_set, x='Survived', y='Age', ax=ax[1])

ax[1].set_xlabel('Survived ("Выжил")')
ax[1].set_ylabel('Возраст')
ax[1].set_title('Диаграммы размаха для возрастов\nвыживших и погибших пассажиров')

plt.show()

Распределения накладываются друг на друга, а на диаграммах размаха есть незначительное количество выбросов. Пока что не будем убирать выбросы, так как масимальное значение возраста пассажира равно 80 годам, что выглядит реалистично.

Выполним t-тест, чтобы окончательно разобраться была ли разница между средним возрастом выживших и средним возрастом погибших пассажиров. Выдвенем следующие нулевую (**H0**) и альтернативную (**H1**) гипотезы:
- **H0** - средний возраст в обеих группах одинаков;
- **H1** - средний возраст групп различается.

Уровень значимости **p** выберем равным **0.95**.

Прежде чем выполнить тест, расчитаем количество наблюдений в каждой группе, среднее значение и стандартное отклонение.

In [ ]:
exploratory_set.groupby('Survived').Age.agg(['count', 'mean', 'std'])

Выборки имеют разный размер, поэтому установим параметр `equal_var=False` при вызове метода `scipy.stats.ttest_ind()`. 

Также отметим, что средние значения выборок различаются, а стандартные отклонения очень близки, но всё же разные.

Выполним t-тест.

In [ ]:
results = ttest_ind(exploratory_set.query('Survived == 0').Age.dropna(),
                    exploratory_set.query('Survived == 1').Age.dropna(),
                    equal_var=False,
                   )

print(f'p-значение: {results.pvalue:.3f}')

alpha = 0.05

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу.')

Значение $p$ очень близко к пороговму. Законен вопрос мог ли получисться такой результат из-за выбросов?

Удалим выбросы, для этого в выборках оставим только пассажиров моложе 70 лет, и повторно выполним тест. После очистки данных от выбросов выведем диаграммы размаха и результат тестирования гипотезы.

In [ ]:
sns.boxplot(data=exploratory_set.query('Age < 70'), x='Survived', y='Age')

plt.xlabel('Survived ("Выжил")')
plt.ylabel('Возраст')
plt.title('Диаграммы размаха для возрастов\nвыживших и погибших пассажиров')

plt.show()


results = ttest_ind(exploratory_set.query('Survived == 0 and Age < 70').Age.dropna(),
                    exploratory_set.query('Survived == 1 and Age < 70').Age.dropna(),
                    equal_var=False,
                   )

print(f'p-значение: {results.pvalue:.3f}')

alpha = 0.05

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу.')

Оценим к каком квантилю относится значение 70 (после удаления пропусков).

In [ ]:
round(percentileofscore(exploratory_set.Age.dropna(), 70) / 100, 2)

**Можно подытожить, что у нас нет оснований утверждать, что средний возраст выживших и погибших отличается, или что фактор возраста имел решающее значение при спасении.**

Возраст непрерывная величина, но мы имеем дело с задачей классификации. Может быть целесообразным разбить всех пассажиров на на возрастные группы. В группы с более высокими номерами будем относить пассажиров, которые старше. При этом для дискретизации будем использовать функцию `pd.qcut()`, чтобы в каждой группе оказалось приблизительно одинаковое количество значений.

In [ ]:
exploratory_set['AgeGroup'] = pd.qcut(exploratory_set.Age, q=7, labels=list(range(7)))

Проверим количество пассажиров в каждой возрастной групе.

In [ ]:
sns.displot(exploratory_set, x='AgeGroup', kind='hist')

plt.title('Количество пассажиров в каждой возрастной группе')
plt.xlabel('Возрастная группа')
plt.ylabel('Количество пассажиров')

plt.show()

In [ ]:
sns.catplot(data=exploratory_set, x='AgeGroup', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших пассажиров в\nкаждой возрастной группе')
plt.xlabel('Возрастная группа')
plt.ylabel('Доля выживших пассажиров')

plt.show()

Разбивка пассажиров на возрастные группы позволяет предположить наличие закономерности между возрастом пассажира и шансом спастись. Попробуем посмотреть сохраняется ли эта закономерность как для мужчин так и для женщин.

In [ ]:
g = sns.catplot(data=exploratory_set, x='AgeGroup', y='Survived', col='Sex', kind='bar', errorbar=None, aspect=0.7)

g.set(xlabel='Возрастная группа', ylabel='Доля выживших')
g.set_titles(col_template="Атрибут Sex = {col_name}")
g.figure.suptitle('Доля выживших пассажиров в каждой возрастной группе')
g.figure.subplots_adjust(top=0.85)

plt.show()

Доля выживших пассажиров среди женщин выше и она практически не меняется взависимости от возрастной группы. Для мужчин просматривается связь между возрастной группой и долей выживших.

#### SibSp

Рассмотрим как распределены значения атрибута `SibSp`.

In [ ]:
sns.displot(data=exploratory_set, x='SibSp', kind='hist')

plt.title('Количество пассажиров для\nкаждого значения атрибута SibSp')
plt.ylabel('Количество пассажиров')

plt.show()

Оценим долю выживших для каждого значения атрибута `SibSp`.

In [ ]:
sns.catplot(data=exploratory_set, x='SibSp', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших для каждого\nзначения атрибута SibSp')
plt.ylabel('Доля выживших')

plt.show()

#### Parch

Рассмотрим как распределены значения атрибута `Parch`.

In [ ]:
sns.displot(exploratory_set, x='Parch', kind='hist')

plt.title('Количество пассажиров для\nкаждого значения атрибута Parch')
plt.ylabel('Количество пассажиров')

plt.show()

Оценим долю выживших для каждого значения атрибута `Parch`.

In [ ]:
sns.catplot(data=exploratory_set, x='Parch', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших для каждого\nзначения атрибута Parch')
plt.ylabel('Доля выживших')

plt.show()

Перспективной может оказаться идея добавить атрибут `Family`, я вляющийся суммой атрибутов `Parch` и `SibSp`.

In [ ]:
exploratory_set['Family'] = exploratory_set.Parch + exploratory_set.SibSp

Построим гистограмму признака `Family`.

In [ ]:
g = sns.displot(data=exploratory_set, x='Family', kind='hist')

plt.ylabel('Число пассажиров на борту Титаника')
plt.title('Гистограмма атрибута Family')

plt.show()

Изучим долю выживших для каждого значения атрибута `Family`

In [ ]:
sns.catplot(data=exploratory_set, x='Family', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших для каждого\nзначения атрибута Family')
plt.ylabel('Доля выживших')

plt.show()

Также можно попробовать добавить атрибут `isFamily`, который будет равен 1, если пассажир путешествовал с семьёй (если атрибут `Family` больше 0) и в противном случае равен 0.

In [ ]:
exploratory_set['isFamily'] = exploratory_set.Family.clip(lower=0, upper=1)

Проверим сколько человек на Титанике путешествовали с семьёй.

In [ ]:
sns.catplot(data=exploratory_set, x='isFamily', kind='count')

plt.title('Количество человек, путешествовавших с семьёй')
plt.ylabel('Количество человек')

plt.show()

Изучим долю выживших в обеих группах.

In [ ]:
sns.catplot(data=exploratory_set, x='isFamily', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших средпассажиров,\nпутешествовавших с семьёй и без')
plt.ylabel('Доля выживших')

plt.show()

#### Fare

Проверим нет ли связи между стоимостью билета и шансом выжить в кораблекрушении. 

Сначала построим гистограмму атрибута `Fare`.

In [ ]:
sns.displot(data=exploratory_set, x='Fare', stat='density', aspect=2, kde=True)

plt.title('Гистограмма признака Fare')
plt.xlabel('Стоимость билета')
plt.ylabel('Плотность вероятности')

plt.show()

Распределение имеет "очень тяжёлый хвост".

Посмотрим как распределена стоимость билетов взависимости от класса.

In [ ]:
sns.catplot(data=exploratory_set, y='Fare', x='Pclass', aspect=2, kind='box')

plt.title('Распределение стоимости билетов в зависимости от класса')
plt.xlabel('Класс (Pclass)')
plt.ylabel('Стоимость билета (Fare)')

plt.show()

Посмотрим есть ли разница между стомостями билетов погибших и выживших для каждого класса.

In [ ]:
sns.catplot(data=exploratory_set, y='Fare', x='Survived', col='Pclass', sharey=False, kind='box', aspect=0.75)

plt.show()

Явной зависимости не просмативается.

Можно попробовать дискретизировать признак `Fare` аналогично тому как это было сделано с признакоа `Age`.

In [ ]:
exploratory_set['FareGroup'] = pd.qcut(exploratory_set.Fare, 6, labels=range(6))

Оценим долю выживших в каждой группе признака `FareGroup`.

In [ ]:
sns.catplot(data=exploratory_set, x='FareGroup', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших для\nкаждого значения FareGroup')
plt.ylabel('Доля выживших')

plt.show()

#### Cabin

Как было отмечено ранее признка `Cabin` 9номер каюты) содержит слишком большое количество уникальных значений и пропусков.

In [ ]:
print(f'Признак Cabin содержит {exploratory_set.Cabin.nunique()} уникальных значений.')
print(f'Признак Cabin содержит {exploratory_set.Cabin.isna().sum()} пропусков.')

Попробуем создать признак `isCabin`, который будет содержать 1, если номер каюты известен, и 0 в противном случае.

In [ ]:
exploratory_set['isCabin'] = exploratory_set.Cabin.isna().astype('int')

Посмотрим различается ли доля выживших в зависимости от значения признака `isCabin`.

In [ ]:
sns.catplot(data=exploratory_set, x='isCabin', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выжившиж для\nкаждого значения isCabin')
plt.ylabel('Доля выживших')

plt.show()

Оказалось, что среди пассажиров, для которых номер каюты известен, доля выживших ниже.

#### Embarked

Посмотрим сколько пассажиров село на Титаник в каждом порту.

In [ ]:
display(exploratory_set.Embarked.value_counts())


sns.displot(data=exploratory_set, x='Embarked', kind='hist')

plt.title('Количество пассажиров в\nзависимоти от порта посадки')
plt.xlabel('Порт')
plt.ylabel('Количество человек')

plt.show()

Больше всего человек село на корабль в Саутгемптоне.

Проверим долю выживших в зависимости от значения признака `Embarked`.

In [ ]:
sns.catplot(data=exploratory_set, x='Embarked', y='Survived', kind='bar', errorbar=None)

plt.title('Доля выживших в зависимости\nот значения признака Embarked')
plt.ylabel('Доля выживших')

plt.show()

### Вывод

1. Присутствует дисбаланс классов
1. Требуется кодирование категориальных признаков
1. Требуется удаление неинформативных признаков
1. Требуется обработка пропусков
1. Требуется стандартизация данных
1. К исследовательскому набору были добавлены новые признаки, информативность которых еще предстоит оценить:
    - `Title`
    - `AgeGroup`
    - `Family`
    - `isFamily`
    - `FareGroup`
    - `isCabin`
1. Возможно наличие корреляции между добавленными и имевшимися признаками

На первый взгляд, очень важными признаками являются класс билета `Pclass` и пол `Sex` пассажира. Могут оказаться полезны признаки `Title`, `AgeGroup`, `isFamily`. Возможно, окажутся информативны `FareGroup` и `isCabin`. Проверим это предположение после обработки данных.

Отметим что женщины имели более высокие шансы спастись, чем мужчины, и что в данных нет явных сведений о том, что более молодые пассажиры имели больше шансов выжить, однако, может оказаться, что некоторые возрастные группы имели больше шансов спастись.

## Обработка данных

### Разработка новых признаков
На этапе исследовательского анализа мы определили список новых признаков, которые могут оказаться информативны. Создадим собственные трансформеры для добавления этих признаков к обучающей выборке внутри пайплайна, напишем конвейер обработки данных и оценим информативность признаков на выходе этого конвейера.

Для кодирования категориальных переменных попробуем использовать целевероятностное кодирование с помощью `category_encoders.target_encoder.TargetEncoder`.

#### `Title`
Создадим трансформер, который будет добавлять признак `Title` к обучающей выборке.

In [ ]:
def add_title(X, y=None):
    title = X.Name.str.extract(pat=r'\b,\s(.+?)\.\s[\b(]?')
    title = pd.Series(title[0], name='Title').str.lower()
    
    title = title.where(title.isin(['mr', 'miss', 'mrs', 'master']), 'aristocratic')
    
    return pd.concat([X, title], axis='columns')

TitleAdder = FunctionTransformer(func=add_title)

#### `AgeGroup` и `FareGroup`
Используем `KBinsDiscretizer` из `sklearn.preprocessing`, чтобы разделить непрерывный признак `Age` на несколько категорий и закодировать их с помощью целевероятностного кодирования (target encoding). Аналогично поступим с признаков `FareGroup`.

Приведём пайплайн, который выполнит эти операции. Предполагается, что приведённый ниже пайплайн - часть `ColumnTransformer` и на вход пайплайна подаются только два признака: `Age` и `FareGroup`. `Age` и `FareGroup` имеют тип `np.float64`, поэтому необходимо указать `TargetEncoder` на эти столбцы, задав атрибут `cols = [0, 1]`.

```Python
Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('discretizer', KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')),
    ('encoder', TargetEncoder(cols=[0, 1])),
])
```

#### `Family`
Создадим трансформер, который добавляет признак `Family` к обучающей выборке.

In [ ]:
def add_family(X, y=None):
    family = X.Parch + X.SibSp
    family.name ='Family'

    return pd.concat([X, family], axis='columns')

FamilyAdder = FunctionTransformer(func=add_family)

#### `isFramily` и `isCabin`
Для добавления атрибута `isFramily` будем использовать `sklearn.preprocessing.Binarizer`.

Признак `Cabin` сам по себе нам не нужен, мы будем заменять его признаком `isCabin`. Будем исользовать `sklearn.impute.MissingIndicator`, чтобы заменить все пропущенные значения единицами, а остальные нулями.

### Оценка информативности признаков

#### Конвейер
Создадим конвейер, выполняющий обработку данных.

In [ ]:
data_prep_target_enc = Pipeline([
    ('title_adder', TitleAdder),
    ('family_adder', FamilyAdder),
    
    ('col_selector', ColumnTransformer([
        ('drop', 'drop', ['PassengerId', 'Name', 'Ticket']),
        ('is_cabin', MissingIndicator(), ['Cabin']),
        ('is_family', Binarizer(), ['Family']),
        ('cat_features', 
         Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                   ('encoder', TargetEncoder()),
                  ]), 
         ['Pclass', 'Sex', 'Embarked', 'Title']
        ),
        ('age_fare', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('discretizer', KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')),
             ('encoder', TargetEncoder(cols=[0, 1])),
         ]), 
         ['Age', 'Fare']
        ),
        ('num_features', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', StandardScaler()),
         ]), 
         ['SibSp', 'Parch', 'Family']
        ),
    ], remainder='passthrough')
    ),        
])

Попробуем обработать с помощью разработанного конвейера обучающую выборку, преобразовать результат в датафрейм и вывести его первые 5 строк.

In [ ]:
cols = [
    'isCabin', 
    'isFamily',
    'Pclass', 
    'Sex', 
    'Embarked', 
    'Title', 
    'AgeGroup', 
    'FareGroup', 
    'SibSp', 
    'Parch',
    'Family'
]

preprocessed_data = pd.DataFrame(data_prep_target_enc.fit_transform(X_train, y_train), columns=cols)
preprocessed_data.head()

#### Корреляция между обучающими признаками
Оценим корреляцию между признаками после обработки данных. Так как в наборе появились качественные признаки (хоть и закодированные), расчитаем коэффициент ранговой корреляции Кендалла.

In [ ]:
plot_corr_matrix(preprocessed_data, vmin=-1, vmax=1, size=(10, 10), method='kendall')

Мы получили скоррелированные между собой признаки. Если появление корреляции, например, между `Famlily` и `SibSp` ожидаемо, то корреляция между параметрами `Title` и `Sex` выглядит неожиданной, но объяснимой: обращения для мужчин и женщин разлины, а женщины имели более высокие шансы выжить.

В данных после обработки нашим конвейером появились признаки с сильной корреляцией, например `Title` и `Sex`, `SibSp` и `Family`. Один из признаков в каждой паре можно будет отбросить, либо применить какую-либо другую технику борьбы с мультиколлинеарностью (например разложение на главные компоненты) или технику отбора признаков.

#### Взаимная информация (Mutual Information)
Чтобы выяснить какие обучающие признаки несут больше всего информации о целевом, оценим метрику Mutual Information с помощью `sklearn.feature_selection.mutual_info_classif`. Оценку будем выполнять дважды при значении параметра `n_neighbors` равном 5 и 100.

In [ ]:
def print_mi_scores(df, n_neighbors):
    mi_scores = (pd
                 .DataFrame(mutual_info_classif(df, 
                                                y_train, 
                                                n_neighbors=n_neighbors, 
                                                random_state=SEED), 
                            columns=['mutual_info'], 
                            index=df.columns
                           )
                 .sort_values(by='mutual_info', ascending=False)
    )

    print(f'n_neighbors={n_neighbors}')
    display(mi_scores)

    (mi_scores
     .sort_values(by='mutual_info', ascending=True)
     .plot(kind='barh', grid=True, title=f'Mutual information\nn_neighbors={n_neighbors}')
    )

    plt.legend(loc='lower right')
    plt.show()
    
    
print_mi_scores(preprocessed_data, 5)

In [ ]:
print_mi_scores(preprocessed_data, 100)

Согласно полученным результатам, признаки неравноценны. Признаки `isFamily`, `Parch`, `SibSp`, `AgeGroup` в обоих случаях попали в 5 пирзнаков с самым низким значение метрики. Возможно, их можно будет безболезненно удалить. Среди 5 признаков с самыми высокими показателями метрики оказались `Title`, `Sex`, `isCabin`, `FareGroup` и `Pclass`.

Прежде чем удалять неинформативные признаки и часть скоррелированных, оценим базовую модель и из нескольких классификаторов выберем лучший.

## Базовая модель
В качестве базовой модели, с которой мы будем сравнивать разработанные модели, будем использовать `sklearn.dummy.DummyClassifier`

Прежде чем приступить к подбору классификатора и оценке моделей, напишем несколько функций, позволяющих быстро их оценить.

In [ ]:
def print_score(estimator, features, target, score='accuracy', cv=5, n_jobs=-1):
    '''Вычисляет среднее, минимальное и максимальное значения метрики,
    полученное с помощью кросс-валидации.
    '''
    
    scores = cross_val_score(estimator,
                             X=features,
                             y=target,
                             cv=cv,
                             scoring=score,
                             n_jobs=n_jobs,
                            )
    
    res = pd.DataFrame(scores, columns=[score]).agg(['mean', 'median', 'min', 'max']).transpose()
    
    scorer = get_scorer(score)
    res.loc[score, 'results_on_train_set'] = scorer(estimator.fit(features, target), features, target)
    
    return res


def plot_roc_curve_for_random_clf():
    '''Выводит на экран кривую ROC для классификатора, предсказывающего
       целевую переменную случайным образом.
    '''
    
    fig, ax = plt.subplots(1, 1)
    
    ax.plot([0,1], [0,1], 'k--', label='Random classifier')
    ax.grid()
    ax.set_xlim((0, 1))
    ax.set_ylim((0, 1))

    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')

    ax.legend(loc='lower right')

    ax.set_title('ROC curve')
    
    return fig, ax


def plot_roc_curve(y_train, y_scores, ax, style='', label=None):
    '''Печатае кривую ROC на оси ax.'''
    
    fpr, tpr, thresholds = roc_curve(y_train, y_scores)
    
    ax.plot(fpr, tpr, style, label=label)
    
    return ax


def evaluate_model(model, X, y, label, ax=None, method='predict_proba', cv=5, n_jobs=-1):
    '''Выводит на экран сводку о производительности модели, состоящую из метрик
    accuracy, f1, roc_auc и кривую ROC. Также возвращат значения метрик в датафрейме df,
    печатает кривую ROC на оси ax.
    '''
    
    df = pd.DataFrame(columns= ['mean', 'median', 'min', 'max'])
    
    for metric in ['accuracy', 'f1', 'roc_auc']:
        df = pd.concat([df, print_score(model, X, y, score=metric, cv=cv, n_jobs=n_jobs)])
        
    print(f'Значения метрик для {label}')
    
    display(df)
        
    y_scores = cross_val_predict(model, X, y, cv=cv, method=method, n_jobs=n_jobs)
    if method == 'predict_proba':
        y_scores = y_scores[:, -1]
    
    if ax is None:
        fig, ax = plot_roc_curve_for_random_clf()        
        ax = plot_roc_curve(y_train, y_scores, label=label, ax=ax)
        plt.show()
        
        return df
    
    else:
        ax = plot_roc_curve(y_train, y_scores, label=label, ax=ax)
        
        return df, ax

In [ ]:
dummy_clf = DummyClassifier(strategy='stratified')

_ = evaluate_model(model=dummy_clf, X=X_train, y=y_train, label='DummyClassifier')

## Выбор модели

Напишем конвейер для которого мы будем подбирать классификатор. Добавим в конвейер апсемплинг, чтобы устранить дисбаланс классов. Шаг, предназначенный для классификатора оставим пустым, так как классификатор еще предстоит подобрать. Также оставим пустым шаг для отбора параметров.

In [ ]:
pipe = Pipeline([
    ('title_adder', TitleAdder),
    ('family_adder', FamilyAdder),
    
    ('col_selector', ColumnTransformer([
        ('drop', 'drop', ['PassengerId', 'Name', 'Ticket']),
        ('is_cabin', MissingIndicator(), ['Cabin']),
        ('is_family', Binarizer(), ['Family']),
        ('cat_features', 
         Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                   ('encoder', TargetEncoder()),
                  ]), 
         ['Pclass', 'Sex', 'Embarked', 'Title']
        ),
        ('age_fare', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('discretizer', KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')),
             ('encoder', TargetEncoder(cols=[0, 1])),
         ]), 
         ['Age', 'Fare']
        ),
        ('num_features', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', StandardScaler()),
         ]), 
         ['SibSp', 'Parch', 'Family']
        ),
    ], remainder='passthrough')
    ),   

    ('smote', SMOTE()),
    
    ('feature_selection', 'passthrough'),
    
    ('classifier', 'passthrough')
])

pipe

Попробуем добавлять к конвейеру разные классификаторы и сравним результаты. Рассмотрим следующие классификаторы с параметрами по умолчанию:
- LogisticRegression;
- SVC;
- KNeighborsClassifier;
- DecisionTreeClassifier;
- RandomFrorestClassifier;
- GradientBoostingClassifier.

In [ ]:
def plot_accuracy(df):
    g = sns.catplot(aspect=1.5)

    g = sns.scatterplot(data=df, 
                      x='classifier', 
                      y='accuracy_on_train_set', 
                      markers='X', 
                      label='Accuracy на тренировочном наборе'
                     )

    g = sns.boxplot(data=df, x='classifier', y='accuracy_on_CV', color='white')

    g.set_ylim([0.6, 1])
    g.tick_params(axis='x', rotation=45)
    g.set_xlabel('Классификатор')
    g.set_ylabel('Accuracy')
    g.set_title('Значения метрики accuracy по результатм\nкросс-валидации для разных классификаторов')
    sns.move_legend(g, 'lower right')

    plt.show()

In [ ]:
classifiers = [
    LogisticRegression(),
    SVC(kernel='rbf'),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
]

methods = [
    'predict_proba',
    'decision_function',
    'predict_proba',
    'predict_proba',
    'predict_proba',
    'predict_proba',
]


fig, ax = plot_roc_curve_for_random_clf()

scores = pd.DataFrame(columns=['classifier', 'accuracy_on_CV', 'accuracy_on_train_set'])

for i, (classifier, method) in enumerate(zip(classifiers, methods)):
    label = type(classifier).__name__
    print(f'{i+1}. {label}')

    pipe.set_params(classifier=classifier)
    
    df, ax = evaluate_model(model=pipe, X=X_train, y=y_train, method=method, label=label, cv=CV, ax=ax)
    
    score = pd.DataFrame(cross_val_score(estimator=pipe, 
                                         X=X_train, 
                                         y=y_train, 
                                         scoring='accuracy',
                                         cv=CV,
                                         n_jobs=-1,
                                        ),
                         columns=['accuracy_on_CV']
                        )
    score['classifier'] = label
    score['accuracy_on_train_set'] = df.loc['accuracy', 'results_on_train_set']
    scores = scores.append(score)
    
    
    print() 
    
plt.legend()
plt.show()

print()


plot_accuracy(scores)

`DecisionTreeClassifier` показал самое низкое значение метрики *ROC_AUC*, далее не будем его рассматривать. Остальные модели показали достаточно хорошие результаты. `SVC(kernel='rbf')`, `RandomForestClassifier` и `GradientBoostingClassifier` показали близкие результаты, при этом медианное значение метрики *accuracy* превысило целевое, либо оказалось очень близко к нему.

На последнем графике, содержащем диаграммы размаха для значений целевой метрики по результатам кросс-валидации, отмечены значения *accuracy*, полученные на тренировочном наборе (модель обучена и оценена на полной обучающей выборке). Значения метрики, полученные на тренировочной выборке, выше мединных значений, полученных на кросс-валидации. Складывается впечатление, что все модели, кроме `LogisticRegression` и `SVC(kernel='rbf')`, демонстрируют оверфитинг.

Самое высокое медианное значение *accuracy* показал`GradientBoostingClassifier`, остановимся на нём. Попробуем встроить в конвейер выбор признаков и настроить гиперпараметры, будем надеяться, что это поможет повысить точность предсказаний и устранить оверфитинг.

## Отбор признаков

Попробуем выбрать информативные признаки с помощью техники называемой *Recursive feature elimination with cross-validation*. Для определения информативности признаков будем использовать `GradientBoostingClassifier` с настройками по умолчанию. 

Определим оптимальное число признаков. В качестве целевой метрики выберем *accuracy*.

In [ ]:
rfecv = RFECV(
    estimator = GradientBoostingClassifier(),
    step=1,
    cv=CV,
    scoring='accuracy',
    min_features_to_select=1,
    n_jobs=-1,
)

print()

rfecv.fit(preprocessed_data, y_train)

print(f"Оптимальное количество признаков: {rfecv.n_features_} из {rfecv.n_features_in_}")

Построим график зависимости среднего значения целевой метрики от количества выбранных признаков, нанесём на график доверительный интервал для каждого значения.

In [ ]:
n_scores = len(rfecv.cv_results_["mean_test_score"])

plt.errorbar(
    x=np.arange(n_scores) + 1,
    y=rfecv.cv_results_["mean_test_score"],
    yerr=rfecv.cv_results_["std_test_score"],
    fmt='s',
    capsize=5,
    ecolor='k',
    elinewidth=1,
)

plt.xlabel("Количество признаков")
plt.ylabel("Средниее значение метрики accuracy")
plt.title("Зависимость метрики accuracy от\nколичества признаков, выбранных RFE")
plt.xlim((0, 12))
plt.ylim((0.7, 0.95))

plt.grid()

plt.show()

Выведем датафрейм с результатом работы алгоритма отбора признаков. Если `Support=False`, значит признак отбрасывается.

In [ ]:
features_info = pd.DataFrame({
    'Name': rfecv.feature_names_in_,
    'Rank': rfecv.ranking_,
    'Support': rfecv.support_
}).sort_values('Rank', ignore_index=True)

features_info

In [ ]:
print_mi_scores(preprocessed_data, 100)

```Python
selector = RFE(estimator=GradientBoostingClassifier(**params), 
               n_features_to_select=(features_info.Rank == 1).sum()
              )

pipe = Pipeline([
    ('title_adder', TitleAdder),
    ('family_adder', FamilyAdder),
    
    ('col_selector', ColumnTransformer([
        ('drop', 'drop', ['PassengerId', 'Name', 'Ticket']),
        ('is_cabin', MissingIndicator(), ['Cabin']),
        ('is_family', Binarizer(), ['Family']),
        ('cat_features', 
         Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                   ('encoder', TargetEncoder()),
                  ]), 
         ['Pclass', 'Sex', 'Embarked', 'Title']
        ),
        ('age_fare', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('discretizer', KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')),
             ('encoder', TargetEncoder(cols=[0, 1])),
         ]), 
         ['Age', 'Fare']
        ),
        ('num_features', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', StandardScaler()),
         ]), 
         ['SibSp', 'Parch', 'Family']
        ),
    ], remainder='passthrough')
    ),   

    ('smote', SMOTE()),
    
    ('feature_selection', selector),
    
    ('classifier', GradientBoostingClassifier())
])

pipe
```

pipe.fit(X_train, y_train)

```Python
scores = pd.DataFrame(columns=['classifier', 'accuracy_on_CV', 'accuracy_on_train_set'])

fig, ax = plot_roc_curve_for_random_clf()

scores = pd.DataFrame(columns=['classifier', 'accuracy_on_CV', 'accuracy_on_train_set'])
    
df, ax = evaluate_model(model=pipe, X=X_train, y=y_train, method='predict_proba', label='pipe', cv=CV, ax=ax)

score = pd.DataFrame(cross_val_score(estimator=pipe, 
                                     X=X_train, 
                                     y=y_train, 
                                     scoring='accuracy',
                                     cv=CV,
                                     n_jobs=-1,
                                    ),
                     columns=['accuracy_on_CV']
                    )
score['classifier'] = 'pipe'
score['accuracy_on_train_set'] = df.loc['accuracy', 'results_on_train_set']
scores = scores.append(score)


print() 
    
plt.legend()
plt.show()

print()


plot_accuracy(scores)
```